In [11]:
from pathlib import Path
with open("dataset.txt", "r", encoding="utf-8") as f:
    text = f.read()

print("Длина текста:", len(text))
print("Первые 500 символов:\n", text[:500])

Длина текста: 807
Первые 500 символов:
 Являясь всего лишь частью общей картины, реплицированные с зарубежных источников, современные исследования своевременно верифицированы. В частности, курс на социально-ориентированный национальный проект однозначно фиксирует необходимость стандартных подходов! Ясность нашей позиции очевидна: повышение уровня гражданского сознания требует анализа форм воздействия. Лишь представители современных социальных резервов, вне зависимости от их уровня, должны быть обнародованы. Современные технологии дост


In [4]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Создаём словарь уникальных символов
chars = sorted(list(set(text)))
char_to_int = {c:i for i,c in enumerate(chars)}
int_to_char = {i:c for i,c in enumerate(chars)}

# Параметры
seq_length = 40  # длина последовательности
step = 3  # шаг сдвига окна

sentences = []
next_chars = []

for i in range(0, len(text) - seq_length, step):
    sentences.append(text[i: i + seq_length])
    next_chars.append(text[i + seq_length])

print("Количество последовательностей:", len(sentences))

# Преобразуем в числа
X = np.zeros((len(sentences), seq_length, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_to_int[char]] = 1
    y[i, char_to_int[next_chars[i]]] = 1


Количество последовательностей: 256


In [5]:
from tensorflow.keras.layers import Input

model = Sequential()
model.add(Input(shape=(seq_length, len(chars))))
model.add(LSTM(128))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_1 (LSTM)                        │ (None, 128)                 │          87,552 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 42)                  │           5,418 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 92,970 (363.16 KB)

 Trainable params: 92,970 (363.16 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
epochs = 20
batch_size = 128

model.fit(X, y, batch_size=batch_size, epochs=epochs)

Epoch 1/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 3.0994
Epoch 2/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 3.0954
Epoch 3/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 3.0915
Epoch 4/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 3.0881
Epoch 5/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 3.0827
Epoch 6/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 3.0828
Epoch 7/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 3.0739
Epoch 8/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 3.0702
Epoch 9/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 3.0653
Epoch 10/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 3.0646
Epoch 11/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 3.0584
Epoch 12/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 3.0536
Epoch 13/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 3.0476
Epoch 14/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 3.0431
Epoch 15/20
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 3.0346
Epoch 16/20
2/2 ━━━━━━━━━━━━━━━━━━

In [9]:
import random

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# Начальная строка
start_index = random.randint(0, len(text) - seq_length - 1)
generated = text[start_index: start_index + seq_length]
print("Seed:", generated)

# Генерация 400 символов
for i in range(400):
    x_pred = np.zeros((1, seq_length, len(chars)))
    for t, char in enumerate(generated[-seq_length:]):
        x_pred[0, t, char_to_int[char]] = 1

    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, temperature=0.8)
    next_char = int_to_char[next_index]

    generated += next_char

print("Generated text:\n", generated)

Seed: альных резервов, вне зависимости от их у
Generated text:
 альных резервов, вне зависимости от их уд ивлбтпнрнянрвуыниысеичоипнзаох ,ытобу,озвынт и т  нсатонтпиинлоыдазя. л уиссзиуисоифнаныоод ооунызннтоьастипдзоымрепте   а тнчнтныеиянотыыныоыыупмловртианданиондтмети,пнсодмявиойтттивуоыа мндомнхвонзып тннноннта йоренннуо р ианитоут ввешВииайинр,м имиымтвснесс  тнвынннс итрпВьпноаииааэниш  иизнан нтеотнент и,иб,дбме.диотайн ф бчцптд тс нфтдунбс поеиыйоевмтпз,еитвтинеоосеы  т,аоиаяи дд йанлиывиьх


In [13]:
Path("gen_result.txt").write_text(generated, encoding="utf-8")

440